In [21]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
from datetime import datetime
from scipy.stats import variation  
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import warnings; warnings.simplefilter('ignore')

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set_style('whitegrid')
import xgboost
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time

from sklearn.model_selection import cross_val_score

%matplotlib inline

In [23]:
def rmse_mid(list_err):
    final = 0
    for i in list_err:
        final = final + (i*i)
    if len(list_err) == 0:
        return 0
    return math.sqrt(final / len(list_err))

In [24]:
def function_rmse_mape(data, var2):
    final_val = 0
    k = 0
    for i in data[var2]:
        final_val = final_val + (i * i)
        k = k + 1
    return math.sqrt(final_val / k)

In [25]:
#Creating State to state code mapping

data = {'State_Name':['Andaman and Nicobar Islands', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'State Unassigned', 'Tamil Nadu', 'Telangana', 'Total', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'], 'State_Code':['AN','AP','AR','AS','BR','CH','CT','DN','DL','GA','GJ','HR','HP','JK','JH','KA','KL','LA','LD','MP','MH','MN','ML','MZ','NL','OR','PY','PB','RJ','SK','UN','TN','TG','TT','TR','UP','UT','WB']} 
  
state_cd_mapping = pd.DataFrame(data) 
state_cd_mapping.head(5)

,State_Name,State_Code
0,Andaman and Nicobar Islands,AN
1,Andhra Pradesh,AP
2,Arunachal Pradesh,AR
3,Assam,AS
4,Bihar,BR


In [26]:
def model_data_creation(state_wise_daily, state_test_daily):
    #Input statewise test data
    state_test_daily = state_test_daily[['Updated On', 'State', 'Total Tested', 'Positive']].rename(columns={'Updated On' : 'Date', 'State' : 'State_Name', 'Total Tested' : 'Tests', 'Positive' : 'Cases' })
    state_test_daily = state_test_daily.merge(state_cd_mapping, on='State_Name', how='left').rename(columns={'State_Code' : 'State'}).drop(columns = ['State_Name'])
    state_test_daily['Date_time'] = pd.to_datetime(state_test_daily['Date'], format = '%d/%m/%Y')
    state_test_daily = state_test_daily.drop(columns=['Date'])
    state_test_daily = state_test_daily.drop_duplicates()
    
    # Input statewise confirmed, recovered & dead cases
    state_wise_daily['Date_time'] = pd.to_datetime(state_wise_daily['Date'], format = "%d-%b-%y")

    # Create base data in STATE X DATE format
    base_data_df = pd.DataFrame()
    base_data_df['Date'] = state_wise_daily.Date_time.unique()
    base_data_df['Key'] = 1
    list_states = state_wise_daily.columns.tolist()
    rm_ilst = ['Date', 'Status', 'TT', 'Date_time']
    for i in rm_ilst:
        list_states.remove(i)
    state_df = pd.DataFrame()
    state_df["State_cd"] = list_states
    state_df["Key"] = 1

    base_data_df = base_data_df.merge(state_df, on='Key', how='left')
    base_data_df = base_data_df.drop(columns = ['Key'])
    
    conf_list = []
    dead_lsit = []
    recv_list = []
    for i in range(0, len(base_data_df['Date'])):
        val = 0
        val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Confirmed')][base_data_df['State_cd'][i]].values[0]
        conf_list.append(val)
    for i in range(0, len(base_data_df['Date'])):
        val = 0
        val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Deceased')][base_data_df['State_cd'][i]].values[0]
        dead_lsit.append(val)
    for i in range(0, len(base_data_df['Date'])):
        val = 0
        val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Recovered')][base_data_df['State_cd'][i]].values[0]
        recv_list.append(val)
    base_data_df['Confirmed'] = conf_list
    base_data_df['Deceased']  = dead_lsit
    base_data_df['Recovered'] = recv_list

    base_data_df = base_data_df.merge(state_test_daily, left_on = ['Date', 'State_cd'], right_on = ['Date_time','State'], how = 'left') 
    base_data_df = base_data_df.drop(columns=['State', 'Date_time'])
    base_data_df = base_data_df.fillna(0)

    base_data_df = base_data_df.sort_values(['State_cd','Date'], ascending = [True, True]).reset_index().drop(columns = ['index'])
    
    tests_list = []
    cases_list = []

    for i in range(0, base_data_df.shape[0]):
        if i == 0:
            test = 0
            case = 0
        else:
            if base_data_df['State_cd'][i] == base_data_df['State_cd'][i-1]:
                test = base_data_df['Tests'][i] - base_data_df['Tests'][i-1]
                case = base_data_df['Cases'][i] - base_data_df['Cases'][i-1]
            else:
                test = 0
                case = 0
        tests_list.append(test)
        cases_list.append(case)

    base_data_df['tests_new'] = tests_list
    base_data_df['cases_new'] = cases_list
    base_data_df = base_data_df.rename(columns = {'tests_new' : 'Tested', 'Tests' : 'Tested_cumulative', 'Cases' : 'Cases_cumulative', 'cases_new' : 'Cases' })
    base_data_df['Positive_Rate'] = base_data_df['Tested'] / base_data_df['Confirmed']
    
    model_data_df = base_data_df.copy()
    confirmed_columns = ['Date', 'State_cd', 'Confirmed']
    tested_columns = ['Date', 'State_cd', 'Tested']
    pos_rate_columns = ['Date', 'State_cd', 'Positive_Rate']

    for i in range(1, 37):
        model_data_df['Day_' + str(i)] = model_data_df["Date"]-pd.to_timedelta(i, unit='D')
    model_data_df.head(5)

    for i in range(1, 37):
        model_data_df = model_data_df.merge(base_data_df[['Date', 'State_cd', 'Tested', 'Confirmed', 'Deceased', 'Recovered', 'Positive_Rate']], left_on = ['Day_' + str(i), 'State_cd'], right_on = ['Date', 'State_cd'], how = 'left' , suffixes = ('','_Day_'+str(i))).drop(columns = ['Date_Day_'+str(i), 'Day_' + str(i)])
        confirmed_columns.append('Confirmed_Day_'+str(i))
        tested_columns.append('Tested_Day_'+str(i))
        pos_rate_columns.append('Positive_Rate_Day_'+str(i))

    model_data_df = model_data_df.fillna(0)

    confirmed_df = model_data_df[confirmed_columns]
    tested_df = model_data_df[tested_columns]
    pos_rate_df = model_data_df[pos_rate_columns]

    return model_data_df

In [27]:
state_wise_daily = pd.read_csv(r'state_wise_daily.csv')
state_test_daily = pd.read_csv(r'statewise_tested_numbers_data.csv')
model_data_df_vb = model_data_creation(state_wise_daily, state_test_daily)
model_data_df_vb.head(10)

,Date,State_cd,Confirmed,Deceased,Recovered,Tested_cumulative,Cases_cumulative,Tested,Cases,Positive_Rate,...,Tested_Day_35,Confirmed_Day_35,Deceased_Day_35,Recovered_Day_35,Positive_Rate_Day_35,Tested_Day_36,Confirmed_Day_36,Deceased_Day_36,Recovered_Day_36,Positive_Rate_Day_36
0,2020-03-14,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-15,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-16,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-17,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-18,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-03-19,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-03-20,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-03-21,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2020-03-22,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2020-03-23,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def modelrun_onestate_alldate(inp_state_cd, inp_start_date, inp_end_date, model_data_dict, inp_depth = 2, inp_estimators = 50, inp_learning_rate = 0.1, inp_min_chld_wt = 5):
   
    y_pred = []
    y_actual = []
    date = []
    state = []
    rmse_error = []
    mape_error = []
    for i in range(1,22):
        this_dataset = model_data_dict[i].copy()
        this_dataset = this_dataset[this_dataset['State_cd'] == inp_state_cd]

        pred_date = inp_end_date + dt.timedelta(days = i)
        if pred_date > datetime(2020, 8, 1):
            break
        df_model_train = this_dataset[(this_dataset['Date'] >= inp_start_date) & (this_dataset['Date'] <= inp_end_date)]
    #     df_model_test  = this_dataset[(this_dataset['Date'] >  inp_mid_date)   & (this_dataset['Date'] <= inp_end_date)]
        df_model_test  = this_dataset[(this_dataset['Date'] == pred_date)]

        X_df_model_train = df_model_train.drop(columns = ['State_cd','Date','Confirmed'])
        X_df_model_test  = df_model_test.drop(columns = ['State_cd','Date','Confirmed'])
        Y_df_model_train = df_model_train['Confirmed']
        Y_df_model_test  = df_model_test['Confirmed']

        xgb_reg = XGBRegressor(max_depth = inp_depth, n_estimators  = inp_estimators, min_child_weight = inp_min_chld_wt, learning_rate = inp_learning_rate, booster = 'gblinear' ,  objective = 'reg:squarederror' )
        xgb_reg.fit(X_df_model_train, Y_df_model_train)

        Train_data_set = df_model_train[['State_cd','Date','Confirmed']]
        Test_data_set  = df_model_test[['State_cd','Date','Confirmed']]

        Train_data_set['preds_train'] = xgb_reg.predict(X_df_model_train)
        Test_data_set['preds_test']  = xgb_reg.predict(X_df_model_test)

        Train_data_set['error_train'] = Train_data_set['preds_train'] - Y_df_model_train
        Test_data_set['error_test']  = Test_data_set['preds_test'] - Y_df_model_test

        y_pred.append(xgb_reg.predict(X_df_model_test)[0])
        y_actual.append(Test_data_set['Confirmed'].tolist()[0])
        state.append(inp_state_cd)
        date.append(pred_date)
        rmse_error.append(np.sqrt(((xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0]) ** 2).mean()))
        mape_error.append(np.mean(np.abs((Test_data_set['Confirmed'].tolist()[0] - xgb_reg.predict(X_df_model_test)[0]) / Test_data_set['Confirmed'].tolist()[0])) )
#         importances_iter = pd.DataFrame({
#             'Feature': X_df_model_train.columns,
#             'Importance': xgb_reg.feature_importances_
#         })
#         importances_iter = importances_iter.sort_values(by='Importance', ascending=False)
#         print(importances_iter)

    #         print(Train_data_set)
    #         print(Test_data_set)

    state_output_df   = pd.DataFrame(list(zip(state, date, y_actual, y_pred, rmse_error, mape_error)), columns = ['State','Date','y_actual','y_pred','RMSE','MAPE'])
    return state_output_df

In [41]:
def daywise_dataCreation2(i, input_model_file):
    day_model = pd.DataFrame()
    cols_to_add_lst = ['Date','State_cd','Confirmed']
    day_model =  input_model_file[cols_to_add_lst]
    list_type_var = ['Tested','Confirmed','Deceased','Recovered','Positive_Rate']
    use_list_var  = ['Confirmed']
    
#     day_model['Day_var'] = 0
#     day_model['Week_var'] = 0
#     min_date = min(day_model['Date']).dayofyear
#     day_model['Day_var'] = day_model['Date'].dt.dayofyear - min_date + 1
#     day_model['Week_var'] = np.floor(day_model['Day_var'] / 7) + 1    
        
    base_3_wt_avg  = 0
    base_5_wt_avg  = 0
    base_7_wt_avg  = 0
    base_9_wt_avg  = 0
    base_11_wt_avg  = 0
    base_13_wt_avg  = 0
    base_14_wt_avg = 0

    for k in use_list_var:
        day_model['3_day_wt_avg_'+k]  = 0
        day_model['5_day_wt_avg_'+k]  = 0
        day_model['7_day_wt_avg_'+k]  = 0
        day_model['9_day_wt_avg_'+k]  = 0
        day_model['11_day_wt_avg_'+k]  = 0
        day_model['13_day_wt_avg_'+k]  = 0
        day_model['14_day_wt_avg_'+k] = 0
        
    for j in range(0,14):
        for k in use_list_var:
            if j < 7 :
                cols_to_add_lst.append(k + '_Day_'+str(i+j))
            if j < 3 :
                day_model['3_day_wt_avg_'+k]  = day_model['3_day_wt_avg_'+k] + ((3-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_3_wt_avg = base_3_wt_avg + (3-j)
            if j < 5 :
                day_model['5_day_wt_avg_'+k]  = day_model['5_day_wt_avg_'+k] + ((5-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_5_wt_avg = base_5_wt_avg + (5-j)
            if j < 7 :
                day_model['7_day_wt_avg_'+k]  = day_model['7_day_wt_avg_'+k] + ((7-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_7_wt_avg = base_7_wt_avg + (7-j)
            if j < 9 :
                day_model['9_day_wt_avg_'+k]  = day_model['9_day_wt_avg_'+k] + ((9-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_9_wt_avg = base_9_wt_avg + (9-j)
            if j < 11 :
                day_model['11_day_wt_avg_'+k]  = day_model['11_day_wt_avg_'+k] + ((11-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_11_wt_avg = base_11_wt_avg + (11-j)
            if j < 13 :
                day_model['13_day_wt_avg_'+k]  = day_model['13_day_wt_avg_'+k] + ((13-j) * input_model_file[k + '_Day_'+str(i+j)])
                base_13_wt_avg = base_13_wt_avg + (13-j)
            day_model['14_day_wt_avg_'+k]  = day_model['14_day_wt_avg_'+k] + ((14-j) * input_model_file[k + '_Day_'+str(i+j)])
            base_14_wt_avg = base_14_wt_avg + (14-j)
        
    for k in use_list_var:
        day_model['3_day_wt_avg_'+k]  = day_model['3_day_wt_avg_'+k] / base_3_wt_avg
        day_model['5_day_wt_avg_'+k]  = day_model['5_day_wt_avg_'+k] / base_5_wt_avg
        day_model['7_day_wt_avg_'+k]  = day_model['7_day_wt_avg_'+k] / base_7_wt_avg
        day_model['9_day_wt_avg_'+k]  = day_model['9_day_wt_avg_'+k] / base_9_wt_avg
        day_model['11_day_wt_avg_'+k]  = day_model['11_day_wt_avg_'+k] / base_11_wt_avg
        day_model['13_day_wt_avg_'+k]  = day_model['13_day_wt_avg_'+k] / base_13_wt_avg
        day_model['14_day_wt_avg_'+k] = day_model['14_day_wt_avg_'+k] / base_14_wt_avg
        
    day_model
    day_model[cols_to_add_lst] = input_model_file[cols_to_add_lst]
    return day_model

In [42]:
model_data_dict2 = {}
model_data_dict2[1] = daywise_dataCreation2(1, model_data_df_vb)
model_data_dict2[2] = daywise_dataCreation2(2, model_data_df_vb)
model_data_dict2[3] = daywise_dataCreation2(3, model_data_df_vb)
model_data_dict2[4] = daywise_dataCreation2(4, model_data_df_vb)
model_data_dict2[5] = daywise_dataCreation2(5, model_data_df_vb)
model_data_dict2[6] = daywise_dataCreation2(6, model_data_df_vb)
model_data_dict2[7] = daywise_dataCreation2(7, model_data_df_vb)
model_data_dict2[8] = daywise_dataCreation2(8, model_data_df_vb)
model_data_dict2[9] = daywise_dataCreation2(9, model_data_df_vb)
model_data_dict2[10] = daywise_dataCreation2(10, model_data_df_vb)
model_data_dict2[11] = daywise_dataCreation2(11, model_data_df_vb)
model_data_dict2[12] = daywise_dataCreation2(12, model_data_df_vb)
model_data_dict2[13] = daywise_dataCreation2(13, model_data_df_vb)
model_data_dict2[14] = daywise_dataCreation2(14, model_data_df_vb)
model_data_dict2[15] = daywise_dataCreation2(15, model_data_df_vb)
model_data_dict2[16] = daywise_dataCreation2(16, model_data_df_vb)
model_data_dict2[17] = daywise_dataCreation2(17, model_data_df_vb)
model_data_dict2[18] = daywise_dataCreation2(18, model_data_df_vb)
model_data_dict2[19] = daywise_dataCreation2(19, model_data_df_vb)
model_data_dict2[20] = daywise_dataCreation2(20, model_data_df_vb)
model_data_dict2[21] = daywise_dataCreation2(21, model_data_df_vb)
model_data_dict2[22] = daywise_dataCreation2(22, model_data_df_vb)

In [43]:
def modelrun_onefunc_all(inp_state_cd, inp_start_date, inp_end_date, model_data_dict, inp_depth, inp_estimators, inp_learning_rate, inp_min_chld_wt):
   
    y_pred = []
    y_actual = []
    date = []
    startdate = []
    state = []
    error = []
    error_7 = []
    error_7_14 = []
    error_14_21 = []
    
    sta = []
    rme = []
    sdt = []
    edt = []
    rm1 = []
    rm2 = []
    rm3 = []

    for i in range(1,22):
        this_dataset = model_data_dict[i].copy()
        this_dataset = this_dataset[this_dataset['State_cd'] == inp_state_cd]

        pred_date = inp_end_date + dt.timedelta(days = i)
#         if pred_date > datetime(2020, 8, 1):
#             break
        df_model_train = this_dataset[(this_dataset['Date'] >= inp_start_date) & (this_dataset['Date'] <= inp_end_date)]
    #     df_model_test  = this_dataset[(this_dataset['Date'] >  inp_mid_date)   & (this_dataset['Date'] <= inp_end_date)]
        df_model_test  = this_dataset[(this_dataset['Date'] == pred_date)]

        X_df_model_train = df_model_train.drop(columns = ['State_cd','Date','Confirmed'])
        X_df_model_test  = df_model_test.drop(columns = ['State_cd','Date','Confirmed'])
        Y_df_model_train = df_model_train['Confirmed']
        Y_df_model_test  = df_model_test['Confirmed']

        xgb_reg = XGBRegressor(max_depth=inp_depth, n_estimators=inp_estimators, min_child_weight=inp_min_chld_wt, learning_rate =inp_learning_rate, booster = 'gblinear' ,  objective = 'reg:squarederror' )
        xgb_reg.fit(X_df_model_train, Y_df_model_train)

        Train_data_set = df_model_train[['State_cd','Date','Confirmed']]
        Test_data_set  = df_model_test[['State_cd','Date','Confirmed']]

        Train_data_set['preds_train'] = xgb_reg.predict(X_df_model_train)
        Test_data_set['preds_test']  = xgb_reg.predict(X_df_model_test)

        Train_data_set['error_train'] = Train_data_set['preds_train'] - Y_df_model_train
        Test_data_set['error_test']  = Test_data_set['preds_test'] - Y_df_model_test

        y_pred.append(xgb_reg.predict(X_df_model_test)[0])
        y_actual.append(Test_data_set['Confirmed'].tolist()[0])
        startdate.append(inp_start_date)
        state.append(inp_state_cd)
        date.append(pred_date)
        error.append(xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0])
#         rmse_error.append(np.sqrt(((xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0]) ** 2).mean()))
#         mape_error.append(np.mean(np.abs((Test_data_set['Confirmed'].tolist()[0] - xgb_reg.predict(X_df_model_test)[0]) / Test_data_set['Confirmed'].tolist()[0])) )

        if i <= 7:
            error_7.append(xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0])
        elif i <= 14:
            error_7_14.append(xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0])
        else:
            error_14_21.append(xgb_reg.predict(X_df_model_test)[0] - Test_data_set['Confirmed'].tolist()[0])


    state_output_df   = pd.DataFrame(list(zip(state, startdate, date, y_actual, y_pred, error)), columns = ['State','startdate', 'Date','y_actual','y_pred','error'])
    rmse_out = function_rmse_mape(state_output_df, 'error')
    edt.append(inp_end_date)
    sta.append(inp_state_cd)
    rme.append(rmse_out)
    sdt.append(inp_start_date)
    rm1.append(rmse_mid(error_7))
    rm2.append(rmse_mid(error_7_14))
    rm3.append(rmse_mid(error_14_21))
    return state_output_df, pd.DataFrame(list(zip(sta, sdt, edt, rme, rm1, rm2, rm3)), columns = ['sta', 'sdt', 'edt', 'rme', 'rm1', 'rm2', 'rm3'])

In [44]:
# inp_start_date = [datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
# inp_end_date   = datetime(2020, 7, 10)
# inp_depth = [1, 2, 3]
# inp_estimators = [50, 100, 150, 200, 300]
# inp_learning_rate = [0.01, 0.02, 0.05, 0.1]
# inp_min_chld_wt = [1, 2, 3, 4, 5]

final_date_output = pd.DataFrame()
final_err_output  = pd.DataFrame()

# list_state_cd = ['AN','AP','AR','AS','BR','CH','CT','DL','DN','GA','GJ','HP','HR','JH','JK','KA','KL','LA','MH','ML','MN','MP','MZ','NL','OR','PB','PY','RJ','SK','TG','TN','TR','UN','UP','UT','WB']

list_state_cd = ['AN','AP','AR','AS','BR','CH','CT','DL','DN','GA','GJ','HP','HR','JH','JK','KA','KL','LA','MH','ML','MN','MP','MZ','NL','OR','PB','PY','RJ','SK','TG','TN','TR','UN','UP','UT','WB']


# inp_start_date = [datetime(2020, 7, 1), datetime(2020, 5, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 5, 1), datetime(2020, 4, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 5, 1), datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
inp_start_date_all = [datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
inp_end_date = [datetime(2020, 8, 7)] 
inp_depth = 1
inp_trees = [50,300,50,50,300,50,50,50,50,50,50,50,50,50,50,50,50,100,300,50,50,50,50,50,150,50,50,50,50,300,50,50,50,300,300,300]
inp_lrate = [0.01,0.1,0.01,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.1,0.01,0.01,0.01,0.05,0.01,0.01,0.02,0.1,0.01,0.01,0.01,0.01,0.01,0.05,0.01,0.01,0.01,0.01,0.1,0.01,0.01,0.02,0.1,0.1,0.1]
inp_mincw = 5

input_df = pd.DataFrame(list(zip(list_state_cd, inp_trees, inp_lrate)), columns = ['State', 'trees', 'lrate'])

for sdt in inp_start_date_all:
    for edt in inp_end_date:
        for i in range(input_df.shape[0]):
            out_df, err_df = modelrun_onefunc_all(input_df.State[i], sdt, edt, model_data_dict2, inp_depth, input_df.trees[i], input_df.lrate[i], inp_mincw)
            final_date_output = final_date_output.append(out_df)
            final_err_output  = final_err_output.append(err_df)
print(final_date_output)
print(final_err_output)

   State  startdate       Date  y_actual       y_pred        error
0     AN 2020-04-01 2020-08-08         0   126.654190   126.654190
1     AN 2020-04-01 2020-08-09         0   141.460541   141.460541
2     AN 2020-04-01 2020-08-10         0   159.411026   159.411026
3     AN 2020-04-01 2020-08-11         0   190.420059   190.420059
4     AN 2020-04-01 2020-08-12         0   222.448395   222.448395
..   ...        ...        ...       ...          ...          ...
16    WB 2020-07-01 2020-08-24         0  3994.612061  3994.612061
17    WB 2020-07-01 2020-08-25         0  4181.296875  4181.296875
18    WB 2020-07-01 2020-08-26         0  4321.010742  4321.010742
19    WB 2020-07-01 2020-08-27         0  4407.250977  4407.250977
20    WB 2020-07-01 2020-08-28         0  4671.745605  4671.745605

[3024 rows x 6 columns]
   sta        sdt        edt           rme           rm1           rm2  \
0   AN 2020-04-01 2020-08-07    414.620596    206.368512    429.362191   
0   AP 2020-04-01 2020-

In [39]:
# final_date_output.to_csv('0729_outall_bestvintage.csv')
# final_err_output.to_csv('0729_errall_bestvintage.csv')

In [44]:
# final_date_output.to_csv('0729_outall_alldate.csv')
# final_err_output.to_csv('0729_errall_alldate.csv')

In [49]:
# final_date_output.to_csv('0729_outall_alldate_wkday.csv')
# final_err_output.to_csv('0729_errall_alldate_wkday.csv')

In [33]:
# final_date_output.to_csv('0730_outall_17July_wkday.csv')
# final_err_output.to_csv('0730_errall_17July_wkday.csv')

# final_date_output.to_csv('0730_outall_17July.csv')
# final_err_output.to_csv('0730_errall_17July.csv')

In [23]:
# final_date_output.to_csv('0802_outall_17July_wkday.csv')
# final_err_output.to_csv('0802_errall_17July_wkday.csv')

# final_date_output.to_csv('0802_outall_17July.csv')
# final_err_output.to_csv('0802_errall_17July.csv')

In [45]:
# final_date_output.to_csv('0807_outall_finalpred_wkday01.csv')
# final_err_output.to_csv('0807_errall_finalpred_wkday01.csv')

final_date_output.to_csv('0807_outall_finalpred01.csv')
final_err_output.to_csv('0807_errall_finalpred01.csv')

FOR 30Th JULY _ ACTUAL PREDICTIONS

In [60]:
# inp_start_date = [datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
# inp_end_date   = datetime(2020, 7, 10)
# inp_depth = [1, 2, 3]
# inp_estimators = [50, 100, 150, 200, 300]
# inp_learning_rate = [0.01, 0.02, 0.05, 0.1]
# inp_min_chld_wt = [1, 2, 3, 4, 5]

final_date_output = pd.DataFrame()
final_err_output  = pd.DataFrame()

list_state_cd = ['AN','AP','AR','AS','BR','CH','CT','DL','DN','GA','GJ','HP','HR','JH','JK','KA','KL','LA','MH','ML','MN','MP','MZ','NL','OR','PB','PY','RJ','SK','TG','TN','TR','UN','UP','UT','WB']

# inp_start_date = [datetime(2020, 7, 1), datetime(2020, 5, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 5, 1), datetime(2020, 4, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 5, 1), datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
inp_start_date_all = [datetime(2020, 4, 1), datetime(2020, 5, 1), datetime(2020, 6, 1), datetime(2020, 7, 1)]
inp_end_date = [datetime(2020, 7, 29)] 
inp_depth = 1
inp_trees = [50,300,50,50,300,50,50,50,50,50,50,50,50,50,50,50,50,100,300,50,50,50,50,50,150,50,50,50,50,300,50,50,50,300,300,300]
inp_lrate = [0.01,0.1,0.01,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.1,0.01,0.01,0.01,0.05,0.01,0.01,0.02,0.1,0.01,0.01,0.01,0.01,0.01,0.05,0.01,0.01,0.01,0.01,0.1,0.01,0.01,0.02,0.1,0.1,0.1]
inp_mincw = 5

input_df = pd.DataFrame(list(zip(list_state_cd, inp_trees, inp_lrate)), columns = ['State', 'trees', 'lrate'])

for sdt in inp_start_date_all:
    for edt in inp_end_date:
        for i in range(input_df.shape[0]):
            out_df, err_df = modelrun_onefunc_all(input_df.State[i], sdt, edt, model_data_dict2, inp_depth, input_df.trees[i], input_df.lrate[i], inp_mincw)
            final_date_output = final_date_output.append(out_df)
            final_err_output  = final_err_output.append(err_df)
print(final_date_output)
print(final_err_output)

   State  startdate       Date  y_actual       y_pred        error
0     AN 2020-04-01 2020-07-30         0    48.737514    48.737514
1     AN 2020-04-01 2020-07-31         0    51.209038    51.209038
2     AN 2020-04-01 2020-08-01         0    56.338520    56.338520
3     AN 2020-04-01 2020-08-02         0    60.099766    60.099766
4     AN 2020-04-01 2020-08-03         0    58.417450    58.417450
..   ...        ...        ...       ...          ...          ...
17    WB 2020-07-01 2020-08-16         0  4498.927734  4498.927734
18    WB 2020-07-01 2020-08-17         0  4541.465820  4541.465820
19    WB 2020-07-01 2020-08-18         0  4488.278320  4488.278320
20    WB 2020-07-01 2020-08-19         0  4668.091309  4668.091309
21    WB 2020-07-01 2020-08-20         0  4928.854980  4928.854980

[3168 rows x 6 columns]
   sta        sdt        edt           rme           rm1           rm2  \
0   AN 2020-04-01 2020-07-29     55.891703     55.979957     44.302066   
0   AP 2020-04-01 2020-

In [61]:
final_date_output.to_csv('0730_outall_30July_wkday.csv')
final_err_output.to_csv('0730_errall_30July_wkday.csv')

# final_date_output.to_csv('0730_outall_30July.csv')
# final_err_output.to_csv('0730_errall_30July.csv')

In [34]:
model_data_dict2[1].head(10)

,Date,State_cd,Confirmed,3_day_wt_avg_Confirmed,5_day_wt_avg_Confirmed,7_day_wt_avg_Confirmed,9_day_wt_avg_Confirmed,11_day_wt_avg_Confirmed,13_day_wt_avg_Confirmed,14_day_wt_avg_Confirmed,Confirmed_Day_1,Confirmed_Day_2,Confirmed_Day_3,Confirmed_Day_4,Confirmed_Day_5,Confirmed_Day_6,Confirmed_Day_7
0,2020-03-14,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-15,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-16,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-17,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-18,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-03-19,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-03-20,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-03-21,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2020-03-22,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2020-03-23,AN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
model_data_df_vb.head(10)

,Date,State_cd,Confirmed,Deceased,Recovered,Tested_cumulative,Cases_cumulative,Tested,Cases,Positive_Rate,...,Tested_Day_34,Confirmed_Day_34,Deceased_Day_34,Recovered_Day_34,Positive_Rate_Day_34,Tested_Day_35,Confirmed_Day_35,Deceased_Day_35,Recovered_Day_35,Positive_Rate_Day_35
0,2020-03-14,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-15,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-16,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-17,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-18,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-03-19,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-03-20,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-03-21,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2020-03-22,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2020-03-23,AN,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
